# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [1]:
!pip install imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    
    make_scorer
)
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression

### 데이터 읽어오기


In [3]:
ROOT_DIR = "data"
RANDOM_STATE = 2022

# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,NaN,NaN,...,7,NaN,NaN,127,NaN,NaN,1,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,NaN,NaN,...,185,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,NaN,NaN,...,10,NaN,NaN,73,NaN,NaN,1,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,NaN,NaN,...,268,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,NaN,NaN,...,121,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,NaN,NaN,...,318,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,NaN,NaN,...,14,NaN,NaN,197,NaN,NaN,1,NaN,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,NaN,NaN,...,1,NaN,NaN,27,NaN,NaN,1,NaN,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,NaN,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal


### 데이터 결측치 확인

In [4]:
# demo data 만들기
train_data_demo = train_data.copy()

In [5]:
# 수치형 컬럼과 문자형 컬럼 분리
numeric_cols = train_data_demo.select_dtypes(include=[np.number]).columns
categorical_cols = train_data_demo.select_dtypes(include=['object']).columns

# 수치형 컬럼의 결측치를 평균으로 대체
train_data_demo[numeric_cols] = train_data_demo[numeric_cols].fillna(train_data_demo[numeric_cols].mean())

# 문자형 컬럼의 결측치는 다른 방법으로 대체하거나 제거
train_data_demo[categorical_cols] = train_data_demo[categorical_cols].fillna('missing')

# 모든 값이 NaN인 열을 식별하고 삭제
train_data_demo = train_data_demo.dropna(axis=1, how='all')  # 모든 값이 NaN인 열 삭제

In [6]:
train_data_demo

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,2.5,-90,...,50.0,91.8,270,50,114.612,19.9,7,127,1,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.600,7.0,185,1,0,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,12.5,90,...,50.0,91.8,270,50,114.612,19.8,10,73,1,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,12.5,90,...,91.8,270.0,50,85,19.900,12.0,268,1,0,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.200,1.0,318,1,0,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,12.5,90,...,50.0,91.8,270,50,114.612,20.5,14,197,1,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,2.5,-90,...,50.0,91.8,270,50,85.000,19.7,1,27,1,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,12.5,90,...,91.8,270.0,50,85,20.100,13.0,117,1,0,Normal


### 언더 샘플링


데이타 불균형을 해결하기 위해 언더 샘플링을 진행합니다.


In [7]:
normal_ratio = 1.0  # 1.0 means 1:1 ratio

df_normal = train_data_demo[train_data_demo["target"] == "Normal"]
df_abnormal = train_data_demo[train_data_demo["target"] == "AbNormal"]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

  Total: Normal: 38156, AbNormal: 2350


target
AbNormal    2350
Normal      2350
Name: count, dtype: int64

### 데이터 분할


In [8]:
df_train, df_val = train_test_split(
    df_concat,
    test_size=0.3,
    stratify=df_concat["target"],
    random_state=RANDOM_STATE,
)


def print_stats(df: pd.DataFrame):
    num_normal = len(df[df["target"] == "Normal"])
    num_abnormal = len(df[df["target"] == "AbNormal"])

    print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}" + f" ratio: {num_abnormal/num_normal}")


# Print statistics
print(f"  \tAbnormal\tNormal")
print_stats(df_train)
print_stats(df_val)

  	Abnormal	Normal
  Total: Normal: 1645, AbNormal: 1645 ratio: 1.0
  Total: Normal: 705, AbNormal: 705 ratio: 1.0


In [9]:
features = []

for col in df_train.columns:
    try:
        df_train[col] = df_train[col].astype(int)
        features.append(col)
    except:
        continue

train_x = df_train[features]
train_y = df_train["target"]

In [10]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)

## 3. 모델 학습


###### 모델 정의 - 0.136 앙상블 기법 적용 -> 0.152 GridSearch 기법 적용


In [11]:
from imblearn.over_sampling import SMOTE

# SMOTE 적용
smote = SMOTE(random_state=RANDOM_STATE)
train_x, train_y = smote.fit_resample(train_x, train_y)

In [12]:
param_grid_rf = {
    'n_estimators': [100,200,300],
    'max_depth': [10,20,30],
    'min_samples_split':[2,5,10]
}

param_grid_svc = {
    'C': [0.1, 1.0, 10],
    #'kernel': ['linear', 'rbf']
}

param_grid_gb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3,5,7]
}

param_grid_logistic = {
    'C': [0.1, 1.0, 10],
    'solver': ['liblinear', 'lbfgs'],
    'max_iter': [1000, 2000, 3000]
}

rf = RandomForestClassifier(random_state=RANDOM_STATE)
svc = SVC(random_state=RANDOM_STATE, probability=True)
gb = GradientBoostingClassifier(random_state=RANDOM_STATE)
logistic = LogisticRegression(random_state=RANDOM_STATE)

f1_scorer = make_scorer(f1_score, pos_label='AbNormal')

grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, scoring=f1_scorer, cv=5, n_jobs=-1)
grid_search_svc = GridSearchCV(estimator=svc, param_grid=param_grid_svc, scoring=f1_scorer, cv=5, n_jobs=-1)
grid_search_gb = GridSearchCV(estimator=gb, param_grid=param_grid_gb, scoring=f1_scorer, cv=5, n_jobs=-1)
grid_search_logistic = GridSearchCV(estimator=logistic, param_grid=param_grid_logistic, scoring=f1_scorer, cv=5, n_jobs=-1)


### 모델 학습


In [13]:
grid_search_rf.fit(train_x, train_y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2022),
             n_jobs=-1,
             param_grid={'max_depth': [10, 20, 30],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring=make_scorer(f1_score, pos_label=AbNormal))

In [14]:
grid_search_svc.fit(train_x, train_y)

GridSearchCV(cv=5, estimator=SVC(probability=True, random_state=2022),
             n_jobs=-1, param_grid={'C': [0.1, 1.0, 10]},
             scoring=make_scorer(f1_score, pos_label=AbNormal))

In [15]:
grid_search_gb.fit(train_x, train_y)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=2022),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200, 300]},
             scoring=make_scorer(f1_score, pos_label=AbNormal))

In [16]:
grid_search_logistic.fit(train_x, train_y)

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=2022), n_jobs=-1,
             param_grid={'C': [0.1, 1.0, 10], 'max_iter': [1000, 2000, 3000],
                         'solver': ['liblinear', 'lbfgs']},
             scoring=make_scorer(f1_score, pos_label=AbNormal))

In [17]:
best_rf = grid_search_rf.best_estimator_
best_svc = grid_search_svc.best_estimator_
best_gb = grid_search_gb.best_estimator_
best_logistic = grid_search_logistic.best_estimator_

ensemble_model = VotingClassifier(estimators=[
    ('rf', best_rf),
    ('svc', best_svc),
    ('gb', best_gb),
    ('logistic', best_logistic)
], voting='soft', n_jobs=-1)

ensemble_model.fit(train_x, train_y)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_depth=20,
                                                     min_samples_split=5,
                                                     random_state=2022)),
                             ('svc', SVC(probability=True, random_state=2022)),
                             ('gb',
                              GradientBoostingClassifier(random_state=2022)),
                             ('logistic',
                              LogisticRegression(C=10, max_iter=1000,
                                                 random_state=2022,
                                                 solver='liblinear'))],
                 n_jobs=-1, voting='soft')

## 4. 제출하기


### 테스트 데이터 예측


테스트 데이터 불러오기


In [32]:
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

In [33]:
# 내가 추가함. NaN값 제거
test_data_demo = test_data.copy()
test_data_demo = test_data_demo.dropna(axis=1, how='all')  # 모든 값이 NaN인 열 삭제

In [34]:
df_test_x = test_data_demo[features]

for col in df_test_x.columns:
    try:
        df_test_x.loc[:, col] = df_test_x[col].astype(int)
    except:
        continue

In [ ]:
test_x_scaled = scaler.transform(df_test_x)

In [36]:
test_pred = ensemble_model.predict(test_x_scaled)
test_pred

array(['AbNormal', 'Normal', 'AbNormal', ..., 'AbNormal', 'AbNormal',
       'Normal'], dtype=object)

### 제출 파일 작성


In [37]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**
